# Phase III: First ML Proof of Concept (5\%)

### Team Names:
- Neel Mallik
- Joshua Herren
- Kunal Lotun
- Kyle Hong

In [1]:
# imports
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#This whole block of code gets the raw data. 
# Site
url = "https://api.rawg.io/api/games"
# API key used to pull data
key = "07dfd979a676418e813cbba97a9dfe4f"
pages = 5
# (1) we'll need something like this since this API limits to the first page in a single get request

# Dictionary of parameters that is sent to the url in a get (r)
params = {
    "key": key,
    "dates": "2024-01-01,2025-12-31",  # filter by date released
    "ordering": "-added",  # sort by added (so we don't get random games with no adds)
    "page": pages
}

# send the get, get the json response, which we can't use directly, so convert it to a dictionary
data = requests.get(url, params=params).json()

# create empty list, we will store all of our data here
games = []
for pages in range(1, pages + 1): #Iterates through each page
    params = {
        "key": key,
        "dates": "2024-01-01,2025-12-31",  # filter by date released
        "ordering": "-added",  # sort by added (so we don't get random games with no adds)
        "page": pages #the current page
    }

    # send the get, get the json response, which we can't use directly, so convert it to a dictionary
    data = requests.get(url, params=params).json()

    for game in data.get("results", []):
        temp = game.get('added_by_status')
        games.append({  # append column with corresponding data (via get("{name of column needed}"))
            "name": game.get("name"),
            # for example, we create the column "name" with data from game (the specific game in "results", which is the actual data from the API, which is, in this case, the name of the game)
            "released": game.get("released"),
            "rating (out of 5.0)": game.get("rating"),
            "ratings_count": game.get("ratings_count"),
            "length (h)": game.get("playtime"),
            "added": game.get("added"),
            #"added_by_status": game.get("added_by_status"),
            "yet to be downloaded": temp.get('yet', 0),
            "owned": temp.get('owned', 0),
            "beaten": temp.get('beaten', 0),
            "have not played yet": temp.get('toplay', 0),
            "dropped": temp.get('dropped', 0),
            "currently playing": temp.get('playing', 0),
            "genres": [genre["name"] for genre in game.get("genres", [])]
            # in this line, we create the column "genres". this is a list of genres (as strings) so we iterate through the list and add    each one to the dataset

        })
# (2) we'll need to create another loop here that iterates however many times "pages" is set to, making sure to create a get request every time it iterates through a single page

# iterate through this until we reach each game
# convert to dataframe, i know this wasn't necessary but it just looked better this way similar to the spotify example
df = pd.DataFrame(games)
# display as a table rather than just text
display(df)

,name,released,rating (out of 5.0),ratings_count,length (h),added,yet to be downloaded,owned,beaten,have not played yet,dropped,currently playing,genres
0,Satisfactory,2024-09-11,4.29,332,12,2894,98,2225,104,147,208,112,"[Indie, Strategy, Adventure, Action]"
1,Vampire: The Masquerade – Bloodlines 2,2025-10-21,3.86,330,329,2428,286,689,28,1370,17,38,"[Action, RPG]"
2,S.T.A.L.K.E.R. 2: Heart of Chornobyl,2024-11-20,3.77,296,7,2150,334,533,106,1067,63,47,"[Shooter, Adventure, Action, RPG]"
3,V Rising,2024-05-08,3.72,169,7,2076,92,1591,71,141,149,32,"[Massively Multiplayer, Adventure, Action]"
4,Hollow Knight: Silksong,2025-09-04,4.29,177,24,1456,174,275,93,822,28,64,"[Indie, Platformer, Adventure, Action]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ANOMALOUS COFFEE MACHINE,2024-08-11,4.08,94,5,266,20,166,3,47,5,25,"[Indie, Simulation]"
96,Rise of the Ronin,2024-03-22,3.33,29,1,263,40,56,23,121,17,6,"[Adventure, Action, RPG]"
97,Still Wakes the Deep,2024-06-18,3.88,85,6,261,22,45,103,67,19,5,"[Adventure, Action]"
98,Silent Hill f,2025-09-24,3.64,58,36,261,32,31,63,108,16,11,"[Adventure, Action]"


In [4]:
# cleaning dataset function
def clean_data(df): 
    """
    Cleans the dataframe by doing the following:
        - Converting times to DateTime
        - Dropping duplicate games
        - Removing invalid rows
        - Showing current player ratio 
        - Turning genres into a string 
        - Adding a popularity score, rentention ratio, and engagement col
        - Removing games that do not meet rating count
        - Adds a Game Number
    
    Args: 
        df (pd.DataFrame) : a data frame to clean

    Returns:
        df (pd.DataFrame) : cleaned data frame
    """

    df = df.drop_duplicates(subset=['name']) # Remove Duplicates
    df = df.dropna(subset=['name', 'rating (out of 5.0)', 'ratings_count', 'added']) # Remove NA rows (rows without names, ratings, rating count, added or not)
    df['released'] = pd.to_datetime(df['released'], errors='coerce') # Converts released to datetime 

    # CLEANING GENRES INTO DUMMY BOOL COLUMNS | https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get_dummies.html | Completely forgot how to use this but I used this StackOverflow article: https://stackoverflow.com/questions/18889588/create-dummies-from-column-with-multiple-values-in-pandas
    genre_dummies = df['genres'].str.join(', ').str.get_dummies(sep=', ').astype(bool) # create boolean columns for each unique genre found in the dataset, separated by ','. For example, "(RPG, Strategy,..)"
    df = df.drop('genres', axis=1).join(genre_dummies) # remove the original genres column and add the new boolean genre columns
    # must clear output and remake the original data (execute above cell) then run this one in order to rerun

    df['active rate'] = (df['currently playing'] / (df['owned'] + 1)).round(3) # Adds the current player ratio
    df['popularity score'] = (df['added'] + df['owned'] + df['currently playing']).round(3) # Adds a pop score, calculated by adding the amount added, owned, and currently playing
    df['retention ratio'] = ((df['currently playing'] + df['beaten']) / (df['dropped'] + 1)).round(3) # Adds a retention score, how well people who bought the game are still playing it
    df['engagement'] = ((df['currently playing'] + df['beaten']) / (df['added'] + 1)).round(3) # Fraction of interested users who actually played or completed the game
    df = df[df['ratings_count'] > 10] # Remove games that have less than 10 ratings
    df.index.name = "Game Number"
    
    return df 

df = clean_data(df)
display(df)

,name,released,rating (out of 5.0),ratings_count,length (h),added,yet to be downloaded,owned,beaten,have not played yet,...,RPG,Racing,Shooter,Simulation,Sports,Strategy,active rate,popularity score,retention ratio,engagement
Game Number,,,,,,,,,,,,,,,,,,,,,
0,Satisfactory,2024-09-11,4.29,332,12,2894,98,2225,104,147,...,False,False,False,False,False,True,0.050,5231,1.033,0.075
1,Vampire: The Masquerade – Bloodlines 2,2025-10-21,3.86,330,329,2428,286,689,28,1370,...,True,False,False,False,False,False,0.055,3155,3.667,0.027
2,S.T.A.L.K.E.R. 2: Heart of Chornobyl,2024-11-20,3.77,296,7,2150,334,533,106,1067,...,True,False,True,False,False,False,0.088,2730,2.391,0.071
3,V Rising,2024-05-08,3.72,169,7,2076,92,1591,71,141,...,False,False,False,False,False,False,0.020,3699,0.687,0.050
4,Hollow Knight: Silksong,2025-09-04,4.29,177,24,1456,174,275,93,822,...,False,False,False,False,False,False,0.232,1795,5.414,0.108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ANOMALOUS COFFEE MACHINE,2024-08-11,4.08,94,5,266,20,166,3,47,...,False,False,False,True,False,False,0.150,457,4.667,0.105
96,Rise of the Ronin,2024-03-22,3.33,29,1,263,40,56,23,121,...,True,False,False,False,False,False,0.105,325,1.611,0.110
97,Still Wakes the Deep,2024-06-18,3.88,85,6,261,22,45,103,67,...,False,False,False,False,False,False,0.109,311,5.400,0.412


In [ ]:
def add_bias_column(X):
    """
    Args:
        X (array): can be either 1-d or 2-d
    
    Returns:
        Xnew (array): the same array, but 2-d with a column of 1's in the first spot
    """
    
    # If the array is 1-d
    if len(X.shape) == 1:
        Xnew = np.column_stack([np.ones(X.shape[0]), X])
    
    # If the array is 2-d
    elif len(X.shape) == 2:
        bias_col = np.ones((X.shape[0], 1))
        Xnew = np.hstack([bias_col, X])
        
    else:
        raise ValueError("Input array must be either 1-d or 2-d")

    return Xnew


In [ ]:
def line_of_best_fit(oldx, y):
    """
    Args:
        X (array): the x pos of the points (1-d) 
        Y (array): the y pos of the points (1-d)
    
    Returns:
        XY (array): The line of best fit
    """
    X = add_bias_column(oldx)
    return np.linalg.inv(X.T.dot(X)).dot(X.T.dot(y))

In [ ]:
def linreg_predict(Xnew, ynew, m):
    """
    Args:
        Xnew: an array, either 1-d or 2-d which includes all the P predictor features, not including bias term
        ynew: a 1-d array which includes all corresponding response values to `Xnew`
        m: a 1-d array of length $p+1$ which contains the coefficients from the `line_of_best_fit` function

    Returns:
        rval (Dict):
            'ypreds': the predicted values from applying `m` to `Xnew`
            'resids': the residuals, the differences between `ynew` and `ypreds`
            'mse': the mean squared error
            'r2': the coefficient of determination ($R^2$) representing the proportion of variability in `ynew` explained by the line of best fit
    """
    rval = {}
    rval['ypreds'] = ypreds(Xnew, m)
    rval['resids'] = resids(ynew, rval['ypreds'])
    rval['mse'] = mse(rval['resids'])
    rval['r2'] = r2(rval['mse'], ynew)
    return rval

def ypreds(Xnew, m):
    e = []
    for x in Xnew:
        e.append(x*m[1] + m[0])
    return np.array(e)

def resids(ynew, ypreds):
    return np.subtract(ynew, ypreds)

def mse(resids):
    return (resids**2).mean()

def r2(mse, y):
    return 1 - (mse/y.var())

## Part 1
(3%) The implementation (using NumPy) of your first ML model as a function call to the cleaned data

In [5]:

import numpy as np

## Part 2
(2%) A discussion of the preliminary results:
   - This may include checking of assumptions, generated plots/tables, measures of fit, or other attributes of the analysis
   - It does not have to be fully correct, but as a proof of concept must demonstrate that the group is close to completing the analysis

## Initial Approach

## Numpy Model


## Ethical Considerations



## Conclusions
